In [1]:
import tensorflow as tf
from efficientnet.tfkeras import EfficientNetB0  # Change the import to EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load pre-trained EfficientNetB0 model without top (fully connected) layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Adjust input shape

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for the new task
num_classes = 5
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Adjust num_classes for your specific task

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=SGD(learning_rate=0.03, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model on a new dataset
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(r'C:\Users\RUBINOS\Desktop\Diabetic Retinopathy\training', target_size=(224, 224), batch_size=64, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(r'C:\Users\RUBINOS\Desktop\Diabetic Retinopathy\testing', target_size=(224, 224), batch_size=64, class_mode='categorical')

model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=30, validation_data=validation_generator, validation_steps=len(validation_generator))

# Evaluate the model on the validation set
evaluation = model.evaluate(validation_generator, steps=len(validation_generator))

# Print the overall accuracy
accuracy = evaluation[1] * 100
print(f'Overall Accuracy: {accuracy:.2f}%')


Found 885 images belonging to 5 classes.
Found 100 images belonging to 5 classes.
Epoch 1/30
14/14 [==============================] - 72s 5s/step - loss: 1.4421 - accuracy: 0.3582 - val_loss: 1.2306 - val_accuracy: 0.4300
Epoch 2/30
14/14 [==============================] - 61s 4s/step - loss: 1.1908 - accuracy: 0.4927 - val_loss: 1.3187 - val_accuracy: 0.5000
Epoch 3/30
14/14 [==============================] - 61s 4s/step - loss: 1.2042 - accuracy: 0.4723 - val_loss: 1.2001 - val_accuracy: 0.4400
Epoch 4/30
14/14 [==============================] - 62s 4s/step - loss: 1.1030 - accuracy: 0.5232 - val_loss: 1.1483 - val_accuracy: 0.4900
Epoch 5/30
14/14 [==============================] - 65s 5s/step - loss: 1.0368 - accuracy: 0.5525 - val_loss: 1.1170 - val_accuracy: 0.5200
Epoch 6/30
14/14 [==============================] - 66s 5s/step - loss: 1.0444 - accuracy: 0.5356 - val_loss: 1.1000 - val_accuracy: 0.5600
Epoch 7/30
14/14 [==============================] - 64s 5s/step - loss: 1.0453

In [2]:
model.save('C:/Users/RUBINOS/Desktop/Diabetic Retinopathy/Untitled12.h5')
print("Model saved as HDFS file.")

C:\Users\RUBINOS\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as HDFS file.
